In [1]:
# Ödev-1: Haber Başlığı Veri Toplama ve Ön İşleme
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta
import time
import re
import string
from collections import Counter

# Türkçe doğal dil işleme için kütüphaneler
from zemberek import TurkishSentenceNormalizer, TurkishMorphology, TurkishTokenizer
from zemberek import TurkishSentenceExtractor

# Gensim ve sklearn kütüphaneleri
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# Görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'DejaVu Sans'

class NewsDataCollector:
    def __init__(self):
        # NewsAPI anahtarı - ücretsiz hesap için günlük limit var
        self.api_key = "281597f62eb44872b2b643b1e79d866c"  # Buraya kendi API anahtarınızı girin
        self.base_url = "https://newsapi.org/v2/everything"
        
    def collect_news_data(self, query_terms=None, days_back=30, max_articles=1000):
        """
        Haber verilerini NewsAPI'den toplar
        """
        if query_terms is None:
            query_terms = ["ekonomi", "siyaset", "spor", "teknoloji", "sağlık", "eğitim", "çevre", "kültür"]
        
        all_articles = []
        
        # Tarih aralığını belirle
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back)
        
        for query in query_terms:
            print(f"'{query}' konusu için veri toplama...")
            
            params = {
                'q': query,
                'language': 'tr',
                'from': start_date.strftime('%Y-%m-%d'),
                'to': end_date.strftime('%Y-%m-%d'),
                'sortBy': 'publishedAt',
                'pageSize': min(100, max_articles // len(query_terms)),
                'apiKey': self.api_key
            }
            
            try:
                response = requests.get(self.base_url, params=params)
                data = response.json()
                
                if response.status_code == 200 and 'articles' in data:
                    for article in data['articles']:
                        if article['title'] and len(article['title'].strip()) > 10:
                            all_articles.append({
                                'title': article['title'],
                                'description': article.get('description', ''),
                                'source': article['source']['name'],
                                'publishedAt': article['publishedAt'],
                                'url': article['url'],
                                'category': query
                            })
                else:
                    print(f"API Hatası: {data.get('message', 'Bilinmeyen hata')}")
                    
            except Exception as e:
                print(f"Hata: {e}")
            
            # API rate limit için bekle
            time.sleep(1)
        
        return pd.DataFrame(all_articles)
    
    def create_sample_data(self, size=1000):
        """
        API anahtarı yoksa örnek veri oluştur
        """
        print("Örnek haber başlığı verisi oluşturuluyor...")
        
        # Türkçe örnek haber başlıkları
        sample_titles = [
            "Ekonomide son gelişmeler ve piyasa analizi",
            "Teknoloji sektöründe yeni yatırımlar açıklandı",
            "Sağlık Bakanlığı'ndan önemli açıklama",
            "Eğitim reformu ile ilgili detaylar belli oldu",
            "Spor müsabakalarında heyecan dorukta",
            "Çevre koruma projelerine büyük destek",
            "Kültür ve sanat etkinlikleri devam ediyor",
            "Siyasi gelişmeler gündem oluşturuyor",
            "Ekonomik büyüme rakamları açıklandı",
            "Teknolojide yapay zeka devrimi",
            "Sağlık sisteminde dijital dönüşüm",
            "Eğitimde uzaktan öğretim modeli",
            "Spor yatırımları artış gösteriyor",
            "Çevre dostu teknolojiler gelişiyor",
            "Kültürel miras korunuyor",
            "Siyasi istikrar ekonomiyi etkiliyor"
        ]
        
        categories = ["ekonomi", "teknoloji", "sağlık", "eğitim", "spor", "çevre", "kültür", "siyaset"]
        sources = ["Haber A", "Haber B", "Haber C", "Gazete X", "Gazete Y", "Portal Z"]
        
        # Varyasyonlarla veri oluştur
        data = []
        for i in range(size):
            base_title = np.random.choice(sample_titles)
            category = np.random.choice(categories)
            source = np.random.choice(sources)
            
            # Başlığa küçük varyasyonlar ekle
            variations = [
                base_title,
                base_title + " - Detaylar",
                base_title + " gündemde",
                base_title + " konuşuluyor",
                "Son dakika: " + base_title,
                base_title + " haberleri",
                base_title + " açıklaması yapıldı"
            ]
            
            title = np.random.choice(variations)
            
            data.append({
                'title': title,
                'description': f"{title} ile ilgili detaylı bilgiler...",
                'source': source,
                'publishedAt': datetime.now().isoformat(),
                'category': category,
                'url': f"https://example.com/haber-{i}"
            })
        
        return pd.DataFrame(data)

class TurkishTextPreprocessor:
    def __init__(self):
        try:
            self.morphology = TurkishMorphology.create_with_defaults()
            self.normalizer = TurkishSentenceNormalizer(self.morphology)
            print("Zemberek başarıyla yüklendi")
        except:
            print("Zemberek yüklenemedi, alternatif yöntemler kullanılacak")
            self.morphology = None
            self.normalizer = None
    
    def clean_text(self, text):
        """
        Metni temizle ve normalize et
        """
        if pd.isna(text):
            return ""
        
        # Küçük harfe çevir
        text = text.lower()
        
        # HTML etiketlerini temizle
        text = re.sub(r'<[^>]+>', '', text)
        
        # URL'leri temizle
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
        
        # Email adreslerini temizle
        text = re.sub(r'\S+@\S+', '', text)
        
        # Sayıları temizle
        text = re.sub(r'\d+', '', text)
        
        # Özel karakterleri temizle (Türkçe karakterler hariç)
        text = re.sub(r'[^\w\s\u00C0-\u017F]', ' ', text)
        
        # Çoklu boşlukları tek boşluğa çevir
        text = re.sub(r'\s+', ' ', text)
        
        return text.strip()
    
    def tokenize_turkish(self, text):
        """
        Türkçe metin için tokenizasyon
        """
        if self.morphology:
            try:
                results = self.morphology.analyze_sentence(text)
                tokens = [result.dictionary_item.lemma for result in results]
                return tokens
            except:
                pass
        
        # Alternatif basit tokenizasyon
        return text.split()
    
    def lemmatize_text(self, text):
        """
        Metni lemmatize et
        """
        if self.morphology:
            try:
                results = self.morphology.analyze_sentence(text)
                lemmatized = [result.dictionary_item.lemma for result in results]
                return ' '.join(lemmatized)
            except:
                pass
        
        # Alternatif: sadece temizlenmiş metin döndür
        return self.clean_text(text)
    
    def stem_text(self, text):
        """
        Basit stemming (Türkçe için yaklaşık)
        """
        # Türkçe için basit stemming kuralları
        common_suffixes = [
            'lar', 'ler', 'da', 'de', 'ta', 'te', 'dan', 'den', 'tan', 'ten',
            'in', 'ın', 'un', 'ün', 'nin', 'nın', 'nun', 'nün',
            'i', 'ı', 'u', 'ü', 'si', 'sı', 'su', 'sü'
        ]
        
        words = text.split()
        stemmed_words = []
        
        for word in words:
            if len(word) > 3:
                for suffix in sorted(common_suffixes, key=len, reverse=True):
                    if word.endswith(suffix) and len(word) - len(suffix) > 2:
                        word = word[:-len(suffix)]
                        break
            stemmed_words.append(word)
        
        return ' '.join(stemmed_words)

def main():
    print("Haber Başlığı Çoğaltma Tespiti - Ödev 1")
    print("=" * 50)
    
    # 1. Veri Toplama
    collector = NewsDataCollector()
    
    # Gerçek API anahtarı varsa NewsAPI'den, yoksa örnek veri
    try:
        df = collector.collect_news_data(max_articles=1000)
        if df.empty:
            print("API'den veri alınamadı, örnek veri oluşturuluyor...")
            df = collector.create_sample_data(1000)
    except:
        print("Örnek veri oluşturuluyor...")
        df = collector.create_sample_data(1000)
    
    print(f"Toplam {len(df)} haber başlığı toplandı")
    
    # 2. Veri Analizi
    print("\nVeri Seti Analizi:")
    print(f"- Toplam kayıt: {len(df)}")
    print(f"- Benzersiz başlık: {df['title'].nunique()}")
    print(f"- Kaynak sayısı: {df['source'].nunique()}")
    
    # 3. Metin Ön İşleme
    preprocessor = TurkishTextPreprocessor()
    
    print("\nMetin ön işleme başlıyor...")
    
    # Temizleme
    df['title_clean'] = df['title'].apply(preprocessor.clean_text)
    
    # Lemmatization
    df['title_lemmatized'] = df['title_clean'].apply(preprocessor.lemmatize_text)
    
    # Stemming
    df['title_stemmed'] = df['title_clean'].apply(preprocessor.stem_text)
    
    # Boş kayıtları temizle
    df = df[df['title_clean'].str.len() > 0]
    df = df[df['title_lemmatized'].str.len() > 0]
    df = df[df['title_stemmed'].str.len() > 0]
    
    print(f"Temizleme sonrası kayıt sayısı: {len(df)}")
    
    # 4. Temizlenmiş veri setlerini kaydet
    lemmatized_df = df[['title', 'title_lemmatized', 'source', 'category']].copy()
    lemmatized_df.columns = ['original_title', 'content', 'source', 'category']
    lemmatized_df.to_csv('lemmatized.csv', index=False, encoding='utf-8')
    
    stemmed_df = df[['title', 'title_stemmed', 'source', 'category']].copy()
    stemmed_df.columns = ['original_title', 'content', 'source', 'category']
    stemmed_df.to_csv('stemmed.csv', index=False, encoding='utf-8')
    
    print("Temizlenmiş veri setleri kaydedildi:")
    print("- lemmatized.csv")
    print("- stemmed.csv")
    
    # 5. TF-IDF Modelleri
    print("\nTF-IDF modelleri oluşturuluyor...")
    
    # Lemmatized için TF-IDF
    tfidf_lem = TfidfVectorizer(
        max_features=5000,
        min_df=2,
        max_df=0.8,
        ngram_range=(1, 2),
        stop_words=None  # Türkçe stop words listesi eklenebilir
    )
    
    tfidf_lem_matrix = tfidf_lem.fit_transform(lemmatized_df['content'])
    
    # Stemmed için TF-IDF
    tfidf_stem = TfidfVectorizer(
        max_features=5000,
        min_df=2,
        max_df=0.8,
        ngram_range=(1, 2),
        stop_words=None
    )
    
    tfidf_stem_matrix = tfidf_stem.fit_transform(stemmed_df['content'])
    
    # TF-IDF modellerini kaydet
    with open('tf-idf_lemmatized.pkl', 'wb') as f:
        pickle.dump({'model': tfidf_lem, 'matrix': tfidf_lem_matrix}, f)
    
    with open('tf-idf_stemmed.pkl', 'wb') as f:
        pickle.dump({'model': tfidf_stem, 'matrix': tfidf_stem_matrix}, f)
    
    print("TF-IDF modelleri kaydedildi")
    
    # 6. Word2Vec Modelleri
    print("\nWord2Vec modelleri oluşturuluyor...")
    
    # Lemmatized için tokenlar
    lem_tokens = [text.split() for text in lemmatized_df['content']]
    
    # Stemmed için tokenlar
    stem_tokens = [text.split() for text in stemmed_df['content']]
    
    # Word2Vec parametreleri
    w2v_params = [
        {'vector_size': 100, 'window': 5, 'sg': 0},  # CBOW, 100d, window=5
        {'vector_size': 100, 'window': 10, 'sg': 0}, # CBOW, 100d, window=10
        {'vector_size': 200, 'window': 5, 'sg': 0},  # CBOW, 200d, window=5
        {'vector_size': 200, 'window': 10, 'sg': 0}, # CBOW, 200d, window=10
        {'vector_size': 100, 'window': 5, 'sg': 1},  # Skip-gram, 100d, window=5
        {'vector_size': 100, 'window': 10, 'sg': 1}, # Skip-gram, 100d, window=10
        {'vector_size': 200, 'window': 5, 'sg': 1},  # Skip-gram, 200d, window=5
        {'vector_size': 200, 'window': 10, 'sg': 1}  # Skip-gram, 200d, window=10
    ]
    
    # Lemmatized Word2Vec modelleri
    for i, params in enumerate(w2v_params):
        model = Word2Vec(
            sentences=lem_tokens,
            vector_size=params['vector_size'],
            window=params['window'],
            min_count=2,
            workers=4,
            sg=params['sg'],
            epochs=10
        )
        
        model_name = f"w2v_lemmatized_{i+1}"
        model.save(f"{model_name}.model")
        print(f"Kaydedildi: {model_name}.model")
    
    # Stemmed Word2Vec modelleri
    for i, params in enumerate(w2v_params):
        model = Word2Vec(
            sentences=stem_tokens,
            vector_size=params['vector_size'],
            window=params['window'],
            min_count=2,
            workers=4,
            sg=params['sg'],
            epochs=10
        )
        
        model_name = f"w2v_stemmed_{i+1}"
        model.save(f"{model_name}.model")
        print(f"Kaydedildi: {model_name}.model")
    
    print("\nÖdev-1 tamamlandı!")
    print("Oluşturulan dosyalar:")
    print("- lemmatized.csv")
    print("- stemmed.csv")
    print("- tf-idf_lemmatized.pkl")
    print("- tf-idf_stemmed.pkl")
    print("- 16 adet Word2Vec modeli (.model dosyaları)")
    
    # Örnek veri gösterimi
    print("\nÖrnek veriler:")
    print("Lemmatized veri örneği:")
    print(lemmatized_df.head())
    print("\nStemmed veri örneği:")
    print(stemmed_df.head())

if __name__ == "__main__":
    main()

Haber Başlığı Çoğaltma Tespiti - Ödev 1
'ekonomi' konusu için veri toplama...
'siyaset' konusu için veri toplama...
'spor' konusu için veri toplama...
'teknoloji' konusu için veri toplama...
'sağlık' konusu için veri toplama...
'eğitim' konusu için veri toplama...
'çevre' konusu için veri toplama...
'kültür' konusu için veri toplama...
Toplam 800 haber başlığı toplandı

Veri Seti Analizi:
- Toplam kayıt: 800
- Benzersiz başlık: 694
- Kaynak sayısı: 19
2025-06-17 17:13:59,605 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 6.267392635345459

Zemberek başarıyla yüklendi

Metin ön işleme başlıyor...
Temizleme sonrası kayıt sayısı: 800
Temizlenmiş veri setleri kaydedildi:
- lemmatized.csv
- stemmed.csv

TF-IDF modelleri oluşturuluyor...
TF-IDF modelleri kaydedildi

Word2Vec modelleri oluşturuluyor...
2025-06-17 17:14:10,225 - gensim.models.word2vec - INFO
Msg: collecting all words and their counts

2025-06-17 17:14:10,225 - gensim.models.word2